# Create Feature Matrix
This notebook serves as a guide to compile a complete feature matrix using various computational tools

( Maybe include figure )

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib as plt
import numpy as np
import os
import sys
import re
import subprocess
import itertools
from tqdm import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

In [3]:
# ROOT_DIR - root directory
ROOT_DIR = os.getcwd()+'/'

# FEATURE_DIR - directory where feature dataframes are saved
DATA_DIR = ROOT_DIR + 'dataframes/'

# FASTA_DIR - directory where fasta files are saved
FASTA_DIR = ROOT_DIR + 'fasta_files/'

# MISC_DIR - directory where all other files are saved
MISC_DIR = ROOT_DIR + 'misc_files/'

# SOFTWARE_DIR - directory where all software is stored
SOFTWARE_DIR = '/home/anand/Documents/HT_Expression/prediction_software/'

# Table of Contents

1. [Initialize Data](#Initialize-Data)
2. [Generate Features](#Generate-Features)
    1. [mRNA Properties](#mRNA-Properties)
        1. [Codon and Nucleotide Composition](#Codon-and-Nucleotide-Composition)
        2. [tRNA Adaptation Index](#tRNA-Adaptation-Index)
        3. [Folding Energy](#Folding-Energy)
    2. [Physical Properties](#Physical-Properties)
    3. [Secondary Structure](#Secondary-Structure)
    4. [Protein Disorder](#Protein-Disorder)
        1. [DisEMBL](#DisEMBL)
        2. [RONN](#RONN)
        3. [DISOPRED3](#DISOPRED3)
3. [Compile Features](#Compile-Features)

# Initialize Data 
[Back to Top](#Table-of-Contents)

In [4]:
prest_file = DATA_DIR+'DF_prest.csv'
DF_raw_prest = pd.read_csv(prest_file,index_col=0)
print 'Number of entries:', len(DF_raw_prest)
DF_raw_prest.head()

Number of entries: 45206


,prest_id,uniprot_id,conc_cf,aa_seq,nt_seq,aa_len
0,140095,G3V3N0,4.3075,IMTAPSSFEQFKVAMNYLQLYNVPDCLEDIQDADCSSSKCSSSASS...,GACAAGCTTGCGGCCGCAATTATGACAGCTCCCTCCAGTTTTGAGC...,139
1,140099,G3V537,2.9154,TYYAWKHELLGSGTCPALPPREVLGMEELEKLPEEQVAEEELECSA...,GACAAGCTTGCGGCCGCAACCTACTATGCCTGGAAGCATGAGCTGC...,144
2,140225,P12724,1.4877,SLHARPPQFTRAQWFAIQHISLNPPRCTIAMRAINNYRWRCKNQNT...,GACAAGCTTGCGGCCGCATCACTCCATGCCAGACCCCCACAGTTTA...,136
3,140235,H0YH02,6.7224,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,GACAAGCTTGCGGCCGCAGCGAGAGCATTAAATGAAAGCAAAAGAG...,123
4,140309,F5GYC5,3.3848,HRKEPGARLEATRGAARPHKQGTKPMITRPSVSQLGEGKCPSSQHL...,GACAAGCTTGCGGCCGCACATCGGAAAGAGCCTGGGGCAAGGCTGG...,124


# Generate Features
[Back to Top](#Table-of-Contents)

## mRNA Properties
[Back to Top](#Table-of-Contents)

In [39]:
DF_mRNA_features = pd.read_csv(DATA_DIR+'DF_prest.csv',index_col=0)

In [40]:
# Directory where MFold outputs are stored
MFOLD_DIR = MISC_DIR+'mfold_out/'

# Directory that contains the tAI software
TAI_DIR = '/home/anand/Documents/HT_Expression/prediction_software/tAI/codonR/'

In [41]:
# Export fasta files for all mRNA sequences and their first 40 nts

sequences = []
sequences40 = []

for i,row in tqdm(DF_mRNA_features.iterrows()):

    name = FASTA_DIR+'nt_sequences/'+str(row.prest_id)+'.fasta'
    name40 = FASTA_DIR+'nt_sequences/'+str(row.prest_id)+'_40nt.fasta'
    
    # Export FASTA files for all mRNA sequences
    if not os.path.isfile(name):
        seq = SeqRecord(Seq(row.nt_seq).transcribe(), 
                        id=str(row.prest_id),description='prEST #'+str(row.prest_id))
        sequences.append(seq)
        with open(name,'w') as f:
            SeqIO.write(seq,f,'fasta')
        
    # Export FASTA files for first 40 nts of all mRNA sequences
    if not os.path.isfile(name40):
        seq40 = SeqRecord(Seq(row.nt_seq).transcribe()[:40], 
                          id=str(row.prest_id),description='prEST #'+str(row.prest_id))
        sequences40.append(seq40)
        with open(name40,'w') as f:
            SeqIO.write(seq40,f,'fasta')

if not os.path.isfile(FASTA_DIR+'all_nt.fasta'):
    with open(FASTA_DIR+'all_nt.fasta','w') as f:
        SeqIO.write(sequences,f,'fasta')
if not os.path.isfile(FASTA_DIR+'all_nt_40.fasta'):
    with open(FASTA_DIR+'all_nt_40.fasta','w') as f:
        SeqIO.write(sequences40,f,'fasta')

### Codon and Nucleotide Composition
[Back to Top](#Table-of-Contents)

The RNA sequences have some extra nucleotides at their head/tail, which need to be removed before selecting the codons

In [42]:
seqs = []
for i,row in tqdm(DF_mRNA_features.iterrows()):
    trans_seq = Seq(row.nt_seq).translate()
    pos = trans_seq.find(row.aa_seq)
    seqs.append(row.nt_seq[pos*3:(pos+len(row.aa_seq))*3])
    
DF_mRNA_features.loc[:,'true_nt_seq'] = seqs

0it [00:00, ?it/s]/home/anand/.virtualenvs/main/local/lib/python2.7/site-packages/Bio/Seq.py:2041: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [44]:
# Generate a list of all codons

all_codons = [''.join(a) for a in itertools.product('ATCG',repeat=3)]

# For each sequence, split the sequence into groups of 3

seq_codons=[]
for i,row in tqdm(DF_mRNA_features.iterrows()):
    seq_codons.append([row.true_nt_seq[j:j+3] for j in range(0, len(row.true_nt_seq), 3)])

# Calculate the fraction of each codon appearing in each sequence
    
counts = [[np.true_divide(seq_codon.count(codon),len(seq_codon)) for codon in all_codons] for seq_codon in seq_codons]

# Initialize the dataframe columns

for codon in all_codons:
    DF_mRNA_features[codon] = None
    
# Add the data to the feature matrix
    
DF_mRNA_features.loc[:,all_codons] = counts

Now we calculate nucleotide composition and GC content

In [45]:
for nuc in 'ATCG':
    DF_mRNA_features.loc[:,'list_nuc_'+nuc] = [np.true_divide(seq.count(nuc),len(seq)) \
                                               for seq in DF_mRNA_features.true_nt_seq]

DF_mRNA_features.loc[:,'GC_content'] = [np.true_divide(seq.count('G') + seq.count('C'),len(seq)) \
                                        for seq in DF_mRNA_features.true_nt_seq]
DF_mRNA_features.loc[:,'GC30'] = [np.true_divide(seq[:30].count('G') + seq[:30].count('C'),len(seq[:30])) \
                                  for seq in DF_mRNA_features.true_nt_seq]

In [46]:
DF_mRNA_features.head()

,prest_id,uniprot_id,conc_cf,aa_seq,nt_seq,aa_len,true_nt_seq,AAA,AAT,AAC,...,GGA,GGT,GGC,GGG,list_nuc_A,list_nuc_T,list_nuc_C,list_nuc_G,GC_content,GC30
0,140095,G3V3N0,4.3075,IMTAPSSFEQFKVAMNYLQLYNVPDCLEDIQDADCSSSKCSSSASS...,GACAAGCTTGCGGCCGCAATTATGACAGCTCCCTCCAGTTTTGAGC...,139,ATTATGACAGCTCCCTCCAGTTTTGAGCAGTTTAAAGTGGCAATGA...,0.057554,0.043165,0.021583,...,0.007194,0.007194,0.014388,0.014388,0.306954,0.287770,0.182254,0.223022,0.405276,0.466667
1,140099,G3V537,2.9154,TYYAWKHELLGSGTCPALPPREVLGMEELEKLPEEQVAEEELECSA...,GACAAGCTTGCGGCCGCAACCTACTATGCCTGGAAGCATGAGCTGC...,144,ACCTACTATGCCTGGAAGCATGAGCTGCTGGGCTCTGGCACCTGCC...,0.006944,0.013889,0.013889,...,0.006944,0.006944,0.027778,0.034722,0.201389,0.192130,0.275463,0.331019,0.606481,0.533333
2,140225,P12724,1.4877,SLHARPPQFTRAQWFAIQHISLNPPRCTIAMRAINNYRWRCKNQNT...,GACAAGCTTGCGGCCGCATCACTCCATGCCAGACCCCCACAGTTTA...,136,TCACTCCATGCCAGACCCCCACAGTTTACGAGGGCTCAGTGGTTTG...,0.007353,0.051471,0.058824,...,0.007353,0.014706,0.000000,0.000000,0.274510,0.259804,0.262255,0.203431,0.465686,0.566667
3,140235,H0YH02,6.7224,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,GACAAGCTTGCGGCCGCAGCGAGAGCATTAAATGAAAGCAAAAGAG...,123,GCGAGAGCATTAAATGAAAGCAAAAGAGTTAATAATGGCAACACGG...,0.040650,0.056911,0.040650,...,0.024390,0.000000,0.016260,0.008130,0.344173,0.205962,0.186992,0.262873,0.449864,0.366667
4,140309,F5GYC5,3.3848,HRKEPGARLEATRGAARPHKQGTKPMITRPSVSQLGEGKCPSSQHL...,GACAAGCTTGCGGCCGCACATCGGAAAGAGCCTGGGGCAAGGCTGG...,124,CATCGGAAAGAGCCTGGGGCAAGGCTGGAGGCCACAAGAGGAGCTG...,0.032258,0.008065,0.008065,...,0.024194,0.016129,0.000000,0.024194,0.309140,0.190860,0.247312,0.252688,0.500000,0.633333


### SD Sequences
[Back to Top](#Table-of-Contents)

Count the number of Shine-Delgarno (SD) and SD-like sequences on both the forward and reverse strands

In [47]:
def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'} 
    bases = list(seq) 
    bases = [complement[base] for base in bases] 
    return ''.join(bases)

def reverse_complement(s):
    return complement(s[::-1])

def count_sd_like(seq):
    sd_like_pat=r'(?=(AGG)|(GGA)|(GAG)|(GGG)|(GGT)|(GTG))'
    revcomp = reverse_complement(seq)
    return [len([m.start() for m in re.finditer(sd_like_pat, seq)]),
            len([m.start() for m in re.finditer(sd_like_pat, revcomp)])]

def count_sd(seq):
    sd_pat = r'(?=(GG.GG.))'
    revcomp = reverse_complement(seq)
    return [len([m.start() for m in re.finditer(sd_pat, seq)]),
            len([m.start() for m in re.finditer(sd_pat, revcomp)])]

In [49]:
sd_like = [count_sd_like(seq) for seq in DF_mRNA_features.true_nt_seq]

DF_mRNA_features['sd_like_fwd'] = [num[0] for num in sd_like]
DF_mRNA_features['sd_like_rev'] = [num[1] for num in sd_like]
DF_mRNA_features['sd_like_fwd_frac'] = np.true_divide(DF_mRNA_features['sd_like_fwd'],
                                                      [len(seq) for seq in DF_mRNA_features.true_nt_seq])
DF_mRNA_features['sd_like_rev_frac'] = np.true_divide(DF_mRNA_features['sd_like_rev'],
                                                      [len(seq) for seq in DF_mRNA_features.true_nt_seq])


sd = [count_sd(seq) for seq in DF_mRNA_features.true_nt_seq]
DF_mRNA_features['sd_seq_fwd'] = [num[0] for num in sd]
DF_mRNA_features['sd_seq_rev'] = [num[1] for num in sd]
DF_mRNA_features['sd_seq_fwd_frac'] = np.true_divide(DF_mRNA_features['sd_seq_fwd'],
                                                     [len(seq) for seq in DF_mRNA_features.true_nt_seq])
DF_mRNA_features['sd_seq_rev_frac'] = np.true_divide(DF_mRNA_features['sd_seq_rev'],
                                                     [len(seq) for seq in DF_mRNA_features.true_nt_seq])

In [50]:
DF_mRNA_features.head()

,prest_id,uniprot_id,conc_cf,aa_seq,nt_seq,aa_len,true_nt_seq,AAA,AAT,AAC,...,GC_content,GC30,sd_like_fwd,sd_like_rev,sd_like_fwd_frac,sd_like_rev_frac,sd_seq_fwd,sd_seq_rev,sd_seq_fwd_frac,sd_seq_rev_frac
0,140095,G3V3N0,4.3075,IMTAPSSFEQFKVAMNYLQLYNVPDCLEDIQDADCSSSKCSSSASS...,GACAAGCTTGCGGCCGCAATTATGACAGCTCCCTCCAGTTTTGAGC...,139,ATTATGACAGCTCCCTCCAGTTTTGAGCAGTTTAAAGTGGCAATGA...,0.057554,0.043165,0.021583,...,0.405276,0.466667,32,28,0.076739,0.067146,2,1,0.004796,0.002398
1,140099,G3V537,2.9154,TYYAWKHELLGSGTCPALPPREVLGMEELEKLPEEQVAEEELECSA...,GACAAGCTTGCGGCCGCAACCTACTATGCCTGGAAGCATGAGCTGC...,144,ACCTACTATGCCTGGAAGCATGAGCTGCTGGGCTCTGGCACCTGCC...,0.006944,0.013889,0.013889,...,0.606481,0.533333,77,56,0.178241,0.129630,10,3,0.023148,0.006944
2,140225,P12724,1.4877,SLHARPPQFTRAQWFAIQHISLNPPRCTIAMRAINNYRWRCKNQNT...,GACAAGCTTGCGGCCGCATCACTCCATGCCAGACCCCCACAGTTTA...,136,TCACTCCATGCCAGACCCCCACAGTTTACGAGGGCTCAGTGGTTTG...,0.007353,0.051471,0.058824,...,0.465686,0.566667,34,52,0.083333,0.127451,1,4,0.002451,0.009804
3,140235,H0YH02,6.7224,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,GACAAGCTTGCGGCCGCAGCGAGAGCATTAAATGAAAGCAAAAGAG...,123,GCGAGAGCATTAAATGAAAGCAAAAGAGTTAATAATGGCAACACGG...,0.040650,0.056911,0.040650,...,0.449864,0.366667,40,21,0.108401,0.056911,2,0,0.005420,0.000000
4,140309,F5GYC5,3.3848,HRKEPGARLEATRGAARPHKQGTKPMITRPSVSQLGEGKCPSSQHL...,GACAAGCTTGCGGCCGCACATCGGAAAGAGCCTGGGGCAAGGCTGG...,124,CATCGGAAAGAGCCTGGGGCAAGGCTGGAGGCCACAAGAGGAGCTG...,0.032258,0.008065,0.008065,...,0.500000,0.633333,41,29,0.110215,0.077957,1,0,0.002688,0.000000


### tRNA Adaptation Index
[Back to Top](#Table-of-Contents)

Run the tAI code on all 45206 prESTs

In [ ]:
command1 = ['perl', TAI_DIR+'codonM', FASTA_DIR+'all_nt.fasta',MISC_DIR+'tAI_files/prEST_tAI.m']
print 'Running Command:'+' '.join(command1) + '\n'

print subprocess.check_output(command1)

command2 = ['Rscript',MISC_DIR+'tAI_files/calc_tAI.R',MISC_DIR+'tAI_files/',TAI_DIR]
print 'Running Command:'+' '.join(command2) + '\n'

print subprocess.check_output(command2)

Now add the tAI as a feature

In [51]:
DF_mRNA_features.loc[:,'tAI'] = pd.read_csv(MISC_DIR+'tAI_files/prEST_tAI.csv')['x']

### Folding Energy
[Back to Top](#Table-of-Contents)

Calculation of all folding energies may take up to 15 hours.

In [18]:
print 'Command: \'mfold SEQ=\''+FASTA_DIR+'nt_sequences/PREST_ID.fasta\' MAX=1\''

os.chdir(MFOLD_DIR)

energy = []
for prest_id in tqdm(DF_mRNA_features.prest_id):

    # Run mRNA folding energy prediction software on full mRNA
    if not os.path.isfile(MFOLD_DIR+str(prest_id)+'.ct'):
        command = 'mfold SEQ=\''+FASTA_DIR+'nt_sequences/'+str(prest_id)+'.fasta\' MAX=1'
        os.system(command)
    
    # Run mRNA folding energy prediction software on first 40 nucleotides
    if not os.path.isfile(MFOLD_DIR+str(prest_id)+'_40nt'+'.ct'):
        command = 'mfold SEQ=\''+FASTA_DIR+'nt_sequences/'+str(prest_id)+'_40nt.fasta\' MAX=1'
        os.system(command)
    
    # Remove non-essential files
    for f in os.listdir(MFOLD_DIR):
        if not f.endswith('.ct') or f.endswith('_1.ct'):
            os.remove(f)

  0%|          | 0/45206 [00:00<?, ?it/s]

Command: 'mfold SEQ='/home/anand/Documents/HT_Expression/ML_HPA/fasta_files/nt_sequences/PREST_ID.fasta' MAX=1'


Parse the output of the folds

In [52]:
for i,row in tqdm(DF_mRNA_features.iterrows()):

    if os.path.isfile(MFOLD_DIR+str(row.prest_id)+'.ct'):
        with open(MFOLD_DIR+str(row.prest_id)+'.ct','r') as f:
            result = f.read()
            energy = re.search('-\d*\.\d*',result)
            if not energy:
                energy = re.search('\d*\.\d*',result)
            DF_mRNA_features.loc[i,'RNA_folding_energy'] = float(energy.group())
            
    if os.path.isfile(MFOLD_DIR+str(row.prest_id)+'_40nt.ct'):
        with open(MFOLD_DIR+str(row.prest_id)+'_40nt.ct','r') as f:
            result = f.read()
            energy = re.search('-\d*\.\d*',result)
            if not energy:
                energy = re.search('\d*\.\d*',result)
            DF_mRNA_features.loc[i,'RNA_40_energy'] = float(energy.group())

In [53]:
DF_mRNA_features.head()

,prest_id,uniprot_id,conc_cf,aa_seq,nt_seq,aa_len,true_nt_seq,AAA,AAT,AAC,...,sd_like_rev,sd_like_fwd_frac,sd_like_rev_frac,sd_seq_fwd,sd_seq_rev,sd_seq_fwd_frac,sd_seq_rev_frac,tAI,RNA_folding_energy,RNA_40_energy
0,140095,G3V3N0,4.3075,IMTAPSSFEQFKVAMNYLQLYNVPDCLEDIQDADCSSSKCSSSASS...,GACAAGCTTGCGGCCGCAATTATGACAGCTCCCTCCAGTTTTGAGC...,139,ATTATGACAGCTCCCTCCAGTTTTGAGCAGTTTAAAGTGGCAATGA...,0.057554,0.043165,0.021583,...,28,0.076739,0.067146,2,1,0.004796,0.002398,0.308574,-111.56,-3.70
1,140099,G3V537,2.9154,TYYAWKHELLGSGTCPALPPREVLGMEELEKLPEEQVAEEELECSA...,GACAAGCTTGCGGCCGCAACCTACTATGCCTGGAAGCATGAGCTGC...,144,ACCTACTATGCCTGGAAGCATGAGCTGCTGGGCTCTGGCACCTGCC...,0.006944,0.013889,0.013889,...,56,0.178241,0.129630,10,3,0.023148,0.006944,0.252830,-192.52,-10.00
2,140225,P12724,1.4877,SLHARPPQFTRAQWFAIQHISLNPPRCTIAMRAINNYRWRCKNQNT...,GACAAGCTTGCGGCCGCATCACTCCATGCCAGACCCCCACAGTTTA...,136,TCACTCCATGCCAGACCCCCACAGTTTACGAGGGCTCAGTGGTTTG...,0.007353,0.051471,0.058824,...,52,0.083333,0.127451,1,4,0.002451,0.009804,0.230369,-113.76,-4.00
3,140235,H0YH02,6.7224,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,GACAAGCTTGCGGCCGCAGCGAGAGCATTAAATGAAAGCAAAAGAG...,123,GCGAGAGCATTAAATGAAAGCAAAAGAGTTAATAATGGCAACACGG...,0.040650,0.056911,0.040650,...,21,0.108401,0.056911,2,0,0.005420,0.000000,0.356271,-97.22,-5.74
4,140309,F5GYC5,3.3848,HRKEPGARLEATRGAARPHKQGTKPMITRPSVSQLGEGKCPSSQHL...,GACAAGCTTGCGGCCGCACATCGGAAAGAGCCTGGGGCAAGGCTGG...,124,CATCGGAAAGAGCCTGGGGCAAGGCTGGAGGCCACAAGAGGAGCTG...,0.032258,0.008065,0.008065,...,29,0.110215,0.077957,1,0,0.002688,0.000000,0.296122,-107.98,-9.00


In [54]:
DF_mRNA_features.to_csv(DATA_DIR+'mRNA_features.csv')

## Physical Properties
[Back to Top](#Table-of-Contents)

In [8]:
DF_physical_features = pd.read_csv(DATA_DIR+'DF_prest.csv',index_col=0)

The BioPython ProtParam quickly calculates many physical properties for any peptide

In [9]:
from Bio.SeqUtils import ProtParam

mw = []
pI= []
arom = []
instab = []
gravy = []
for seq in tqdm(DF_physical_features.aa_seq):
    data = ProtParam.ProteinAnalysis(seq)
    mw.append(data.molecular_weight())
    pI.append(data.isoelectric_point())
    arom.append(data.aromaticity())
    instab.append(data.instability_index())
    gravy.append(data.gravy())
    
DF_physical_features['bio_pI'] = pI
DF_physical_features['bio_mW'] = mw
DF_physical_features['bio_aromaticity'] = arom
DF_physical_features['bio_instability'] = instab
DF_physical_features['bio_gravy'] = gravy

100%|██████████| 45206/45206 [00:12<00:00, 3730.07it/s]


Get the counts of each amino acid

In [10]:
amino_acids = 'ARNDCQEGHILKMFPSTWYV'
counts = [[np.true_divide(seq.count(aa),len(seq)) for seq in DF_physical_features.aa_seq] for aa in amino_acids]
for i in range(len(amino_acids)):
    DF_physical_features['list_comp_'+amino_acids[i]] = counts[i]

Calculate the charge of each prEST

In [11]:
DF_physical_features['charge'] = [seq.count('K')+seq.count('R')-seq.count('D')-seq.count('E') for seq in DF_physical_features.aa_seq]
DF_physical_features['abs_charge']=abs(DF_physical_features['charge'])
DF_physical_features['avg_charge']=np.true_divide(DF_physical_features['charge'],[len(seq) for seq in DF_physical_features['aa_seq']])
DF_physical_features['abs_avg_charge'] = abs(DF_physical_features['avg_charge'])

Calculate the presence of various amino acid types

In [12]:
DF_physical_features['frac_aliphatic'] = sum([DF_physical_features['list_comp_'+aa] for aa in 'AGILPV'])
DF_physical_features['frac_uncharged_polar'] = sum([DF_physical_features['list_comp_'+aa] for aa in 'STNQ'])
DF_physical_features['frac_polar'] = sum([DF_physical_features['list_comp_'+aa] for aa in 'QNHSTYCMW'])
DF_physical_features['frac_hydrophobic'] = sum([DF_physical_features['list_comp_'+aa] for aa in 'AGILPVF'])
DF_physical_features['frac_positive'] = sum([DF_physical_features['list_comp_'+aa] for aa in 'HKR'])
DF_physical_features['frac_sulfur'] = sum([DF_physical_features['list_comp_'+aa] for aa in 'CM'])
DF_physical_features['frac_negative'] = sum([DF_physical_features['list_comp_'+aa] for aa in 'DE'])
DF_physical_features['frac_amide'] = sum([DF_physical_features['list_comp_'+aa] for aa in 'NQ'])
DF_physical_features['frac_alcohol'] = sum([DF_physical_features['list_comp_'+aa] for aa in 'ST'])

Save the dataframe

In [13]:
DF_physical_features.to_csv(DATA_DIR+'physical_features.csv')

## Secondary Structure
[Back to Top](#Table-of-Contents)

The SCRATCH-1D predictor is computationally intensive and may take weeks to run

In [60]:
DF_ss_features = pd.read_csv(DATA_DIR+'DF_prest.csv',index_col=0)

In [61]:
# Export fasta files for all sequences

sequences = []

for i,row in tqdm(DF_ss_features.iterrows()):

    name = FASTA_DIR+'aa_sequences/'+str(row.prest_id)+'.fasta'
    
    # Export FASTA files for all mRNA sequences
    if not os.path.isfile(name):
        seq = SeqRecord(Seq(row.aa_seq), id=str(row.prest_id),description='prEST #'+str(row.prest_id))
        sequences.append(seq)
        with open(name,'w') as f:
            SeqIO.write(seq,f,'fasta')
        
if not os.path.isfile(FASTA_DIR+'all_prEST.fasta'):
    with open(FASTA_DIR+'all_prEST.fasta','w') as f:
        SeqIO.write(sequences,f,'fasta')

In [62]:
scratch_cmd = '/home/anand/Documents/HT_Expression/prediction_software/SCRATCH-1D_1.1/bin/run_SCRATCH-1D_predictors.sh'
name = FASTA_DIR+'all_prEST.fasta'
out = MISC_DIR+'scratch_out/all_prEST'
n_cores = '8'

print 'Running command:',scratch_cmd,name,out,n_cores

if not os.path.isfile(out+'.ss'):
    subprocess.call([scratch_cmd,name,out,n_cores])

Running command: /home/anand/Documents/HT_Expression/prediction_software/SCRATCH-1D_1.1/bin/run_SCRATCH-1D_predictors.sh /home/anand/Documents/HT_Expression/ML_HPA/fasta_files/all_prEST.fasta /home/anand/Documents/HT_Expression/ML_HPA/misc_files/scratch_out/all_prEST 8


The SCRATCH-1D predictor returns 4 outputs:
* .ss: 3-letter secondary structure
* .ss8: 8-letter secondary structure
* .acc: Solvent accessible residues
* .acc20: Solvent accessibility index

In [63]:
## SS

ss_list = []
for record in SeqIO.parse(MISC_DIR+'scratch_out/all_prEST.ss','fasta'):
    ss_list.append([int(record.id),str(record.seq)])
df_ss = pd.DataFrame(ss_list,columns=['prest_id','SCRATCH_ss'])
DF_ss_features = pd.merge(DF_ss_features,df_ss,on='prest_id')

## SS8
ss8_list = []
for record in SeqIO.parse(MISC_DIR+'scratch_out/all_prEST.ss8','fasta'):
    ss8_list.append([int(record.id),str(record.seq)])
df_ss8 = pd.DataFrame(ss8_list,columns=['prest_id','SCRATCH_ss8'])
DF_ss_features = pd.merge(DF_ss_features,df_ss8,on='prest_id')

## ACC
acc_list = []
for record in SeqIO.parse(MISC_DIR+'scratch_out/all_prEST.acc','fasta'):
    acc_list.append([int(record.id),str(record.seq)])
df_acc = pd.DataFrame(acc_list,columns=['prest_id','SCRATCH_acc'])
DF_ss_features = pd.merge(DF_ss_features,df_acc,on='prest_id')

In [64]:
## ACC20
acc20_list = []
with open(MISC_DIR+'scratch_out/all_prEST.acc20','r') as f:
    result = f.read()
    lines = result[1:-1].split('\n>')
    for line in lines:
        [name,nums] = line.split('\n')
        acc20_mean = np.mean([int(a) for a in nums.split(' ')])
        seq_id = int(name[:name.find(' ')])
        acc20_list.append([seq_id,acc20_mean])
df_acc20 = pd.DataFrame(acc20_list,columns=['prest_id','acc20_mean'])
DF_ss_features = pd.merge(DF_ss_features,df_acc20,on='prest_id')

Construct features based on the SCRATCH-1D output

In [66]:
ss_names = ['ss_helix','ss_ext','ss_c']
ss_letters = 'HEC'
ss8_names = ['ss8_helix','ss8_ext','ss8_turn','ss8_helix3',
             'ss8_pi_helix','ss8_bridge','ss8_bend','ss8_coil']
ss8_letters = 'HETGIBSC'

for i in range(3):
    DF_ss_features[ss_names[i]] = [np.true_divide(ss.count(ss_letters[i]),len(ss)) \
                                   for ss in DF_ss_features.SCRATCH_ss]

for i in range(8):
    DF_ss_features[ss8_names[i]] = [np.true_divide(ss.count(ss8_letters[i]),len(ss)) \
                                    for ss in DF_ss_features.SCRATCH_ss8]
    
DF_ss_features['acc_frac'] = [np.true_divide(seq.count('e'),len(seq)) \
                              for seq in DF_ss_features.SCRATCH_acc]

Calculate fraction of hydrophilic and hydrophobic solvent accessible and inaccessible residues

In [67]:
hydrophilic='RDEKSNQ'
hydrophobic='WFYLIVMACHT'

for i,row in tqdm(DF_ss_features.iterrows()):
    acc_hydrophobic_out = 0
    acc_hydrophilic_out = 0
    acc_hydrophobic_in = 0
    acc_hydrophilic_in = 0
    for j in xrange(len(row.aa_seq)):
        if row.aa_seq[j] in hydrophobic:
            if row.SCRATCH_acc[j]=='e':
                acc_hydrophobic_out+=1
            else:
                acc_hydrophobic_in+=1
        elif row.aa_seq[j] in hydrophilic:
            if row.SCRATCH_acc[j]=='e':
                acc_hydrophilic_out+=1
            else:
                acc_hydrophilic_in+=1
    DF_ss_features.loc[i,'acc_hydrophobic_out'] = np.true_divide(acc_hydrophobic_out,row.aa_len)
    DF_ss_features.loc[i,'acc_hydrophobic_in'] = np.true_divide(acc_hydrophobic_in,row.aa_len)
    DF_ss_features.loc[i,'acc_hydrophilic_out'] = np.true_divide(acc_hydrophilic_out,row.aa_len)
    DF_ss_features.loc[i,'acc_hydrophilic_in'] = np.true_divide(acc_hydrophilic_in,row.aa_len)

Calculate the average GRAVY of solvent accessible and inaccessible residues.

kd is a dictionary that contains the Kyle-Doolittle index of hydrophobicity used to calculate GRAVY

In [68]:
from Bio.SeqUtils.ProtParamData import kd

for i,row in tqdm(DF_ss_features.iterrows()):
    out_gravy = 0
    in_gravy = 0
    for j in xrange(len(row.aa_seq)):
        if row.SCRATCH_acc[j] == 'e':
            out_gravy += kd[row.aa_seq[j]]
        else:
            in_gravy += kd[row.aa_seq[j]]
    DF_ss_features.loc[i,'out_gravy'] = np.true_divide(out_gravy,len(row.aa_seq))
    DF_ss_features.loc[i,'in_gravy'] = np.true_divide(in_gravy,len(row.aa_seq))

In [69]:
DF_ss_features.to_csv(DATA_DIR+'secondary_structure.csv')

## Protein Disorder
[Back to Top](#Table-of-Contents)

In [13]:
DF_disorder_features = pd.read_csv(DATA_DIR+'DF_prest.csv',index_col=0)

### DisEMBL
[Back to Top](#Table-of-Contents)

Run DisEMBL on all prESTs

In [14]:
disembl_cmd = '/home/anand/Documents/HT_Expression/prediction_software/DisEMBL-1.4/DisEMBL.py'

In [16]:
DF_disorder_features['disembl_COILS'] = None
DF_disorder_features['disembl_REM465'] = None
DF_disorder_features['disembl_HOTLOOPS'] = None

disembl_list = []

for prest_id in tqdm(DF_disorder_features.prest_id):
    out = subprocess.check_output([disembl_cmd,'8','8','4','1.2','1.4','1.2',
                                   FASTA_DIR+'aa_sequences/'+str(prest_id)+'.fasta']).split('\n')
    disembl_list.append([out[9],out[11],out[13]])
    
DF_disorder_features[['disembl_COILS','disembl_REM465','disembl_HOTLOOPS']] = disembl_list

In [17]:
import string

def count_upper(seq):
    return len(filter(lambda x: x in string.uppercase, seq))

In [41]:
DF_disorder_features.loc[:,'disembl_COILS_frac'] = [np.true_divide(count_upper(seq),len(seq)) for seq in DF_disorder_features.disembl_COILS]
DF_disorder_features.loc[:,'disembl_REM465_frac'] = [np.true_divide(count_upper(seq),len(seq)) for seq in DF_disorder_features.disembl_REM465]
DF_disorder_features.loc[:,'disembl_HOTLOOPS_frac'] = [np.true_divide(count_upper(seq),len(seq)) for seq in DF_disorder_features.disembl_HOTLOOPS]

### RONN
[Back to Top](#Table-of-Contents)

In [19]:
RONN_DIR = '/home/anand/Documents/HT_Expression/prediction_software/RONN'

Run RONN Command on all prESTs

In [20]:
if not os.path.isfile(MISC_DIR+'RONN_out/out.txt'):
    print 'Running command:',' '.join(['bash',MISC_DIR+'RONN_out/ronn_script.sh',FASTA_DIR+'aa_sequences',RONN_DIR])
    subprocess.call(['bash',MISC_DIR+'RONN_out/ronn_script.sh',FASTA_DIR+'aa_sequences',RONN_DIR])

Parse the output

In [21]:
ronn_results = {}
ronn_avg={}
with open(MISC_DIR+'RONN_out/out.txt', 'rb') as f:
    entries = f.read().split('>')[1:]
    for seq in tqdm(entries):
        seq_list = seq.split('\n')
        title = seq_list[0][:seq_list[0].find(' ')]
        disorder = [aa.split('\t') for aa in seq_list[1:] if aa!='']
        disorder = [[aa[0],float(aa[1])] for aa in disorder]
        ronn_avg[title] = np.mean([aa[1] for aa in disorder])
        ronn_results[title] = ''.join([aa[0].upper() if aa[1]>.5 else aa[0].lower() for aa in disorder])

Add the features to the dataframe

In [48]:
for i,row in tqdm(DF_disorder_features.iterrows()):
    DF_disorder_features.loc[i,'ronn_avg'] = ronn_avg[str(row.prest_id)]
    DF_disorder_features.loc[i,'ronn_results'] = ronn_results[str(row.prest_id)]

DF_disorder_features['ronn_frac'] = [np.true_divide(count_upper(seq),len(seq)) for seq in DF_disorder_features.ronn_results]

In [49]:
DF_disorder_features.head()

,prest_id,uniprot_id,conc_cf,aa_seq,nt_seq,aa_len,disembl_COILS,disembl_REM465,disembl_HOTLOOPS,disembl_COILS_frac,...,disembl_HOTLOOPS_frac,ronn_avg,ronn_results,ronn_frac,disopred_results,disopred_avg,disopred_pb_results,disopred_pb_avg,disopred_frac,disopred_pb_frac
0,140095,G3V3N0,4.3075,IMTAPSSFEQFKVAMNYLQLYNVPDCLEDIQDADCSSSKCSSSASS...,GACAAGCTTGCGGCCGCAATTATGACAGCTCCCTCCAGTTTTGAGC...,139,imtapssfeqfkvamnylqLYNVPDCLEDIQDADCSSSKCSSSASS...,imtapssfeqfkvamnylqlynvpdclediqDADCSSSKCSSSASS...,IMTAPSSFeqfkvamnylqlynvpdclediqdadcSSSKCSSSASS...,0.769784,...,0.410072,0.495827,imtapssfeqfkvamnylqlynvpdclediqDADCSSSKCSSSASS...,0.482014,ImtapssfeqfkvamnylqlynvpdclediqdadcsSSKCSSSASS...,0.387050,-...................................^^^^^^^^^^...,0.317770,0.388489,0.359712
1,140099,G3V537,2.9154,TYYAWKHELLGSGTCPALPPREVLGMEELEKLPEEQVAEEELECSA...,GACAAGCTTGCGGCCGCAACCTACTATGCCTGGAAGCATGAGCTGC...,144,TYYAWKHELLGSGTCPALPPREVLGMEELEKLPEEQvaeeelecsa...,tyyawkhellgsgtcpalpprevlgmeeleklpEEQVAEEELecsa...,TYYAWKHELLGSGTCPALPPrevlgmeeleklpeeqvaeeelecsa...,0.791667,...,0.388889,0.488125,tyyawkhellGSGTcPALPPREVLGMEELEKLPEEQVAEEELECSa...,0.493056,tyyawkhellgsgtCPALPPREVLGMEELEKLPEEQVAEEELECSA...,0.421597,..............^^^^^^--^^^^^^^^^^^^^^^^^^^-^^^^...,0.352917,0.465278,0.423611
2,140225,P12724,1.4877,SLHARPPQFTRAQWFAIQHISLNPPRCTIAMRAINNYRWRCKNQNT...,GACAAGCTTGCGGCCGCATCACTCCATGCCAGACCCCCACAGTTTA...,136,SLHARPPQFTraqwfaiqHISLNPPRCTiamrainnyrwrcknqnt...,slharppqftraqwfaiqhislnpprctiamrainnyrwrcknqnt...,SLHARPPQftraqwfaiqhislnpprctiamrainnyrwrcknqnt...,0.720588,...,0.338235,0.368750,SLharppqftraqwfaiqhislnpprctiamrainnyrwrcknqnt...,0.058824,slharppqftraqwfaiqhislnpprctiamrainnyrwrcknqnt...,0.034338,.................................................,0.000000,0.000000,0.000000
3,140235,H0YH02,6.7224,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,GACAAGCTTGCGGCCGCAGCGAGAGCATTAAATGAAAGCAAAAGAG...,123,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,0.617886,...,0.552846,0.571382,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,0.536585,ARALNESKRVNNGNTAPEDSSPAKKTRRCQRQESKKMPVAGGKANK...,0.425366,^^^^^^^^^^^^^^^-^---^-^^^^^^^^^^^^^--^^^^^^^^^...,0.295122,0.430894,0.373984
4,140309,F5GYC5,3.3848,HRKEPGARLEATRGAARPHKQGTKPMITRPSVSQLGEGKCPSSQHL...,GACAAGCTTGCGGCCGCACATCGGAAAGAGCCTGGGGCAAGGCTGG...,124,HRKEPGARLEATRGAARPHKQGTKPMITRPSVSQLGEGKCPSSQHL...,HRKEPGARLEATRGAARPHKQGTKPMitrpSVSQLGEGKCPSSQHL...,HRKEPGARLEATRGAARPHKQGTKPMITRPSVSQlgegkcpssqhl...,0.693548,...,0.274194,0.556613,HRKEPGARLEATRGAARPHKQGTKPMITRPSVSQLGEGKCPSSQHL...,0.620968,HRKEPGARLEATRGAARPHKQGTkpmitrpsvsqlgegkcpssqhl...,0.236613,^^^^^^^^^^^^^^^^^^^^^^^^.........................,0.185726,0.201613,0.209677


### DISOPRED3
[Back to Top](#Table-of-Contents)

Run DISOPRED3 on all prESTs

**ANAND: (may need to fix script)**

In [24]:
DISOPRED_DIR = MISC_DIR+'disopred_out/'

disopred_cmd='/home/anand/Documents/HT_Expression/prediction_software/DISOPRED/run_disopred.pl'

#subprocess.call(['bash',MISC_DIR+'disopred_out/disopred_script.sh',FASTA_DIR+'aa_sequences',disopred_cmd])

Parse DISOPRED disorder predictions

In [25]:
for i,row in tqdm(DF_disorder_features.iterrows()):
    with open(DISOPRED_DIR+str(row.prest_id)+'.diso', 'rb') as f:
        lines = f.readlines()
        frac = []
        disorder_string = ''
        for line in lines:
            if not line.startswith('#'):
                if line[8] == '*':
                    disorder_string += line[6].upper()
                else:
                    disorder_string += line[6].lower()
                frac.append(float(line[10:14]))
        DF_disorder_features.loc[i,'disopred_results'] = disorder_string
        DF_disorder_features.loc[i,'disopred_avg'] = np.mean(frac)

Parse DISOPRED protein binding predictions

In [26]:
for i,row in tqdm(DF_disorder_features.iterrows()):
    with open(DISOPRED_DIR+str(row.prest_id)+'.pbdat', 'rb') as f:
        lines = f.readlines()
        frac = []
        disorder_string = ''
        for line in lines:
            if not line.startswith('#'):
                disorder_string += line[8]
                try:
                    frac.append(float(line[10:]))
                except:
                    frac.append(0)
        DF_disorder_features.loc[i,'disopred_pb_results'] = disorder_string
        DF_disorder_features.loc[i,'disopred_pb_avg'] = np.mean(frac)

Calculate numerical values

In [46]:
DF_disorder_features.loc[:,'disopred_frac'] = [np.true_divide(count_upper(seq),len(seq)) for seq in DF_disorder_features.disopred_results]
DF_disorder_features.loc[:,'disopred_pb_frac'] = [np.true_divide(seq.count('^'),len(seq)) for seq in DF_disorder_features.disopred_pb_results]

In [50]:
DF_disorder_features.to_csv(DATA_DIR+'protein_disorder.csv')

# Compile Features
[Back to Top](#Table-of-Contents)

In [14]:
DF_prest = pd.read_csv(DATA_DIR+'DF_prest.csv',index_col=0)
DF_mRNA_features = pd.read_csv(DATA_DIR+'mRNA_features.csv',index_col=0)
DF_physical_features = pd.read_csv(DATA_DIR+'physical_features.csv',index_col=0)
DF_ss_features = pd.read_csv(DATA_DIR+'secondary_structure.csv',index_col=0)
DF_disorder_features = pd.read_csv(DATA_DIR+'protein_disorder.csv',index_col=0)

In [15]:
DF_prest_features = DF_prest.merge(DF_mRNA_features).merge(DF_physical_features).merge(DF_ss_features).merge(DF_disorder_features)

In [16]:
split_size = len(DF_prest_features)/4

DF_prest_features[:split_size].to_csv(DATA_DIR+'DF_prest_features_1.csv')
DF_prest_features[split_size:split_size*2].to_csv(DATA_DIR+'DF_prest_features_2.csv')
DF_prest_features[split_size*2:split_size*3].to_csv(DATA_DIR+'DF_prest_features_3.csv')
DF_prest_features[split_size*3:].to_csv(DATA_DIR+'DF_prest_features_4.csv')